In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False
rcParams["font.family"] = 'DejaVu Sans'
rcParams["font.size"] = 12

In [ ]:
def get_positions(row, chosen_sweeps, min_distance=0, end_distance=7):
    
    # the end positions is more robust than the starting position.
    second_half = row.positions[row.positions.shape[0]//2:, :]
    
    try:
        end_position = second_half[second_half[:, 2] < 0.1, :][0, :2]
    except:
        print('did not detect end position with height. did drone land?')
        end_position = row.positions[-1, :2]
        print('using:', end_position)
    
    if row.appendix in ["", "_30"]:
        END_POS = np.array([0.0, 0.3])
    elif row.appendix in ["_50"]:
        print("using 0.5")
        END_POS = np.array([0.0, 0.5])
    else: # crashed into wall
        END_POS = np.array([0.0, end_distance * 1e-2])
        
    delta = end_position + END_POS
    positions_cm = np.c_[delta[0] - row.positions[chosen_sweeps, 0],
                         delta[1] - row.positions[chosen_sweeps, 1], 
                         row.positions[chosen_sweeps, 2]] * 1e2
    
    if np.any(positions_cm[:, 2] < 30):
        return None
    
    mask_ok = positions_cm[:, 1] > min_distance
    chosen_sweeps = chosen_sweeps[mask_ok]
    positions_cm = positions_cm[mask_ok, :]
    return positions_cm

In [ ]:
def get_calib_function(calib="stepper"):
    from calibration import get_calibration_function_median
    from calibration import get_calibration_function_moving
    
    if calib == "manual":
        print("using manual")
        calib_function_median, freqs = get_calibration_function_moving(
            "2021_07_27_manual", motors="all45000", fit_one_gain=False
        )
    elif calib == "flying":
        print("using flying")
        calib_function_median, freqs = get_calibration_function_moving(
            "2021_07_14_flying", motors="linear_buzzer_cont", fit_one_gain=False,
            appendix_list=["_17"]
        )
    elif calib == "stepper":
        print("using stepper dataset")
        calib_function_median, freqs = get_calibration_function_median(
            "2021_07_08_stepper_fast",
            motors="all45000",
            mic_type="audio_deck",
            snr=5,
            fit_one_gain=False,
        )
    return calib_function_median

# Calibration study

In [ ]:
from calibration import get_calibration_function_median
from plotting_tools import save_fig

from matplotlib import cm
cmap = cm.get_cmap('inferno')

MICS = [0, 1]
XLIM = [3000, 4500]
YLIM = [2, 80]
YTICKS = [2, 5, 10, 50]
SIZE = [5, 5]

exp_name = "2021_07_08_stepper_fast"; snr=5

results_df = pd.read_pickle(f"../experiments/{exp_name}/all_data.pkl")
results_df = results_df.loc[results_df.bin_selection == 5]

for motors in [0, "all45000"]:
    flag = "no" if motors == 0 else ""
    fname = f"plots/experiments/calibration_stepper_{flag}motors"

    fig, ax = plt.subplots()
    fig.set_size_inches(*SIZE)
    calib_function_median, freqs = get_calibration_function_median(
        exp_name, mic_type="audio_deck", snr=snr, motors=motors, ax=ax
    )
    #ax.set_title(f"stepper, with {flag} motors")
    ax.legend(loc='upper left')
    ax.set_yscale("log")
    ax.set_xlim(*XLIM)
    ax.set_ylim(*YLIM)
    save_fig(fig, fname + ".pdf")
    
    rows = results_df.loc[results_df.motors == motors]
    
    fig, axs = plt.subplots(1, len(MICS), sharey=True)
    fig.set_size_inches(5*len(MICS), 5)
    print("plottting", len(rows))
    number = len(rows)/2
    for mic_i in MICS:
        label = f"distance {rows.iloc[0].distance}"
        for i, (_, row) in enumerate(rows.iterrows()):
            if i % 2 == 0:
                continue
            stft = row.stft[0]
            freq = row.frequencies_matrix[0]
            #for i, (stft, freq) in enumerate(zip(row.stft, row.frequencies_matrix)):
            color = cmap(i / (1.1 * number))
            axs[mic_i].plot(
                freq[freq > 0],
                np.abs(stft[mic_i, freq > 0]),
                color=color,
                alpha=0.8,
                label=label,
            )
            label = None
    axs[mic_i].set_yscale("log")
    label = f"distance {rows.iloc[-1].distance}"
    for m in MICS:
        axs[m].plot(
            freq[freq > 0],
            np.abs(stft[m, freq > 0]),
            color=color,
            alpha=0.8,
            label=label,
        )
        axs[m].legend(loc="upper right")
        axs[m].set_xlim(*XLIM)
        axs[m].set_xlabel("frequency [Hz]")
        axs[m].grid()
        axs[m].set_title(f"mic{m}")
    axs[0].set_ylim(*YLIM)
    axs[0].set_yticks(YTICKS)
    axs[0].set_yticklabels(YTICKS)
    axs[0].set_ylabel("amplitude")
    save_fig(fig, fname + "_variance.pdf")

In [ ]:
from calibration import get_calibration_function_moving

#name = "manual"
name = "flying"

if name == "manual":
#appendix_list=["", "_2", "_3", "_4"]; motors_list = [0, "all45000"];
    exp_name = "2021_07_27_manual"; motors_list=[0, "all45000"]; appendix_list=["_3"]

elif name == "flying":
#appendix_list=["_17", "_18", "_19"]
    exp_name = "2021_07_14_flying"; motors_list = ["linear_buzzer_cont"]; appendix_list=["_17"]

results_df = pd.read_pickle(f"../experiments/{exp_name}/all_data.pkl")

for motors in motors_list:
    flag = "no" if motors == 0 else ""
    fname = f"plots/experiments/calibration_{name}_{flag}motors"
    
    fig_total, ax_total = plt.subplots()
    fig_total.set_size_inches(*SIZE)
    get_calibration_function_moving(exp_name, motors=motors, fit_one_gain=False, ax=ax_total, 
                                    appendix_list=appendix_list)
    ax_total.set_yscale("log")
    ax_total.set_ylim(*YLIM)
    ax_total.set_xlim(*XLIM)
    save_fig(fig_total, fname + ".pdf")
    
    rows = results_df.loc[(results_df.motors==motors) & 
                          (results_df.appendix.isin(appendix_list))]
    for i_row, row in rows.iterrows():
        fig, axs = plt.subplots(1, len(MICS), sharey=True)
        fig.set_size_inches(5*len(MICS), 5)
        for mic_i in MICS:
            
            #if np.any(np.abs(row.stft[:, mic_i, :]) > 30):
            #    print(f'mic{mic_i}{row.appendix}: problematic')
            #else:
            #    print(f'mic{mic_i}{row.appendix}: ok')
            
            delta = np.r_[1.0, row.positions[1:, 1] - row.positions[:-1, 1]]
            delta[np.isnan(delta)] = 0.0 
            valid_mask = (row.positions[:, 2] > 0.1) & (delta > 0)
            number = np.sum(valid_mask)
            print("plotting", number)
            label = f'time {np.where(valid_mask)[0][0]}'
            for i, (stft, freq) in enumerate(zip(row.stft[valid_mask], row.frequencies_matrix[valid_mask])):
                color = cmap(i/(1.1*number))
                axs[mic_i].plot(freq[freq > 0], np.abs(stft[mic_i, freq>0]), color=color, alpha=0.8, label=label)
                label = None
            label = f'time {i}'
            axs[mic_i].set_yscale("log")
            axs[mic_i].plot(freq[freq > 0], np.abs(stft[mic_i, freq>0]), color=color, alpha=0.8, label=label)
            axs[mic_i].legend(loc='upper right')
            axs[mic_i].grid()
            axs[mic_i].set_xlabel('frequency [Hz]')
            axs[mic_i].set_title(f"mic{mic_i}")
        axs[0].set_ylim(*YLIM)
        axs[0].set_yticks(YTICKS)
        axs[0].set_yticklabels(YTICKS)
        axs[0].set_ylabel("amplitude")
        save_fig(fig, fname + "_variance.pdf")
            
        #ax_total.plot(freq[freq > 0], np.abs(stft_average[mic_i, freq>0]), color=f"C{mic_i}")
        #ax_total.grid()
        #ax_total.legend()

# 3 by 3 study

### use stepper motor data

In [ ]:
from crazyflie_description_py.experiments import WALL_ANGLE_DEG_STEPPER, WALL_DISTANCE_CM_STEPPER
exp_name = "2021_07_08_stepper_fast"
calib = "stepper"

azimuth_deg = WALL_ANGLE_DEG_STEPPER

distance_nom = 30 #30 #47
stft_list = []
freqs_list = []
pos_list = []
mics = [0, 1]
start_i = 0 # for naming only
n_sweeps_per = 3

results_df = pd.read_pickle(f"../experiments/{exp_name}/all_data.pkl")

for distance in np.arange(distance_nom-2,distance_nom+2):
    d_corr = distance - WALL_DISTANCE_CM_STEPPER
    row = results_df.loc[(results_df.distance == d_corr) &
                          (results_df.bin_selection == 5) &
                          (results_df.motors == "all45000"), :].iloc[0]

    label = f"stepper at {distance}"

    stft_list += list(row.stft)[:n_sweeps_per]
    freqs_list += list(row.frequencies_matrix)
    pos_list += list(np.array([[0, distance]] * row.stft.shape[0]))
    print(len(stft_list))
    
n_sweeps = len(stft_list)

fname = f"plots/experiments/{exp_name}_by3_distance{distance_nom}"

### use hovering data

In [ ]:
from crazyflie_description_py.experiments import WALL_ANGLE_DEG

exp_name = "2021_07_27_hover"; 
distance = 50
calib = "moving"

azimuth_deg = WALL_ANGLE_DEG

results_df = pd.read_pickle(f"../experiments/{exp_name}/all_data.pkl")
row = results_df.loc[results_df.appendix == f"_{distance}", :].iloc[0]

label = f"hover at {distance}"
mics = [0, 1]

# choose three sweeps to plot
start_i = 3
n_sweeps = 10

#for start_i in np.arange(row.stft.shape[0]-n_sweeps)[::3]:
chosen_sweeps = np.arange(start_i, start_i+n_sweeps)
stft_list = row.stft[chosen_sweeps]
freqs_list = row.frequencies_matrix[chosen_sweeps]
pos_list = get_positions(row, chosen_sweeps) 
dist_average = np.mean([p[1] for p in pos_list])

fname = f"plots/experiments/{exp_name}_by3_distance{distance}"

In [ ]:
from estimators import DistanceEstimator
from inference import Inference, eps_normalize
from plotting_tools import save_fig, pcolorfast_custom
from matplotlib import cm
import scipy

cmap = cm.get_cmap("inferno")
distance_range = [0.0, 80.0]
distances = np.arange(100)

# for start_i in np.arange(row.stft.shape[0]-n_sweeps-1)[::3]:
#    chosen_sweeps = np.arange(start_i, start_i+n_sweeps)
#    stft_list = row.stft[chosen_sweeps]
#    freqs_list = row.frequencies_matrix[chosen_sweeps]
#    pos_list = get_positions(row, chosen_sweeps)
#    if pos_list is None: # invalid position
#        continue
dist_average = np.mean([p[1] for p in pos_list])

inf_machine = Inference()
inf_machine.add_calibration_function(calib_function_median)
inf_machine.add_geometry(distance_range, azimuth_deg)

fig_slice, ax_slice = plt.subplots(1, len(mics), sharey=True)
fig_slice.set_size_inches(3 * len(mics), 3)

fig_prob, ax_prob = plt.subplots(1, len(mics), sharey=True)
fig_prob.set_size_inches(3 * len(mics), 3)

fig_pos, ax_pos = plt.subplots()
fig_pos.set_size_inches(3, 3)
fig_total, ax_total = plt.subplots()
fig_total.set_size_inches(3, 3)

#### Treat all stfts individually
inf_machine = Inference()
inf_machine.add_calibration_function(calib_function_median)
inf_machine.add_geometry(distance_range, azimuth_deg)

distance_estimator_all = DistanceEstimator()

stft_matrix = np.empty((len(stft_list), len(mics), stft_list[0].shape[1]))
prob_matrix = np.empty((len(stft_list), len(mics), len(distances)))
print(stft_matrix.shape, prob_matrix.shape)


label = f'probability sweep 0'
for i, (stft, freqs, pos) in enumerate(zip(stft_list, freqs_list, pos_list)):
    color = cmap(i / (1.1 * len(stft_list)))  #f"C{i}"

    ax_pos.scatter(pos[0], pos[1], s=20.0, marker="x", color=color)

    inf_machine.add_data(np.abs(stft), freqs)
    inf_machine.filter_out_freqs()

    distance_estimator = DistanceEstimator()

    stft_matrix[i, :, :] = np.abs(stft[mics, :])

    for mic_i, mic_idx in enumerate(mics):
        dist, proba, diff = inf_machine.do_inference("bayes", mic_idx)

        interpolator = scipy.interpolate.interp1d(dist,
                                                  proba,
                                                  kind="linear",
                                                  fill_value="extrapolate")
        proba_int = interpolator(distances)
        prob_matrix[i, mic_i, :] = proba_int

        distance_estimator.add_distribution(diff * 1e-2, proba, mic_idx)
        distance_estimator_all.add_distribution(diff * 1e-2, proba, mic_idx)

    dist, proba = distance_estimator.get_distance_distribution(
        method="sum", distances_m=1e-2 * np.arange(*distance_range, step=1.0)
    )
    ax_total.plot(dist * 1e2,
                  proba,
                  #eps_normalize(proba, 1e-3),
                  color=color,
                  ls='-',
                  label=label,
                  alpha=0.5)
    label = None
label = f'probability sweep {i}'
ax_total.plot(dist * 1e2,
              proba, 
              #eps_normalize(proba, 1e-3),
              color=color,
              ls='-',
              label=label,
              alpha=0.5)

for mic_i in range(len(mics)):

    pcolorfast_custom(
        ax_slice[mic_i],
        freqs[freqs > 0],
        np.arange(len(stft_list)),
        stft_matrix[:, mic_i, freqs > 0],
        n_xticks=3,
        n_yticks=5,
    )

    pcolorfast_custom(
        ax_prob[mic_i],
        distances,
        np.arange(len(stft_list)),
        prob_matrix[:, mic_i, :],
        n_xticks=3,
        n_yticks=5,
    )

dist, proba = distance_estimator_all.get_distance_distribution(
    method="sum",
    distances_m=1e-2 * np.arange(*distance_range, step=1.0),
    verbose=True)
ax_total.plot(
    dist * 1e2,
    proba,
    #eps_normalize(proba, 1e-3),
    color="k",
    ls="-",
    label="averaged probabilities",
)

#### Create average of stfts. DOESN'T GIVE GOOD RESULTS
# stft_array = np.array(stft_list)
# stft = np.median(stft_array, axis=0)
#
# inf_machine.add_data(np.abs(stft), freqs)
#
# freqs = freqs_list[0]
# distance_estimator = DistanceEstimator()
# for mic_i, mic_idx in enumerate(mics):
#     amps = np.abs(stft[mic_idx, :])
#     dist, proba, diff = inf_machine.do_inference("bayes", mic_idx)
#     distance_estimator.add_distribution(diff * 1e-2, proba, mic_idx)
# dist, proba = distance_estimator.get_distance_distribution(
#     method="sum", distances_m=1e-2 * np.arange(*distance_range, step=1.0)
# )
# ax_total.plot(
#     dist * 1e2, eps_normalize(proba, 1e-3), color="k", ls="--", label="averaged sweeps"
# )

#### Plot decorations
ax_prob[0].set_ylabel("sweep number")
ax_slice[0].set_ylabel("sweep number")
for m, mic_idx in enumerate(mics):
    ax_prob[m].set_xlabel("distance [cm]")
    ax_prob[m].set_title(f"mic{m}")

    ax_slice[m].set_xlabel("frequency [Hz]")
    ax_slice[m].set_title(f"mic{m}")

ax_pos.set_xlabel("x [cm]")
ax_pos.set_ylabel("y [cm]")
ax_pos.axis("equal")
ax_pos.grid()

ax_total.set_xlabel("distance [cm]")
ax_total.set_ylabel("probability")
ax_total.axvline(dist_average, ls=":", color="k", label="average distance")
ax_total.legend(loc="upper left", bbox_to_anchor=[1.0, 1.0])
ax_total.grid()
save_fig(fig_pos, f"{fname}_start{start_i}_n{n_sweeps}_pos.pdf")
save_fig(fig_slice, f"{fname}_start{start_i}_n{n_sweeps}_data.pdf")
save_fig(fig_prob, f"{fname}_start{start_i}_n{n_sweeps}_prob.pdf")
save_fig(fig_total, f"{fname}_start{start_i}_n{n_sweeps}.pdf")

# Full study

In [ ]:
from crazyflie_description_py.experiments import WALL_ANGLE_DEG

#calib_method = "flying" # not so good
calib_method = "stepper" # better

# this is good:
#exp_name = "2021_07_27_hover"; 
#appendix = "_30"; motors="hover_sweep"

# this is good:
exp_name = "2021_07_14_flying"; end_distance=7
appendix = "_18"; motors="linear_buzzer_cont";  # 19 is not good because of positioning

# doesn't work:
#exp_name = "2021_07_14_flying_hover"; end_distance=50
#appendix = "_2"; motors="multi_sweep";  

# not so good: 
#exp_name = "2021_07_27_manual"; end_distance=15
#appendix = "_4"; motors=0 #"all45000"

azimuth_deg = WALL_ANGLE_DEG

results_df = pd.read_pickle(f"../experiments/{exp_name}/all_data.pkl")
print(results_df.motors.unique())

row = results_df.loc[(results_df.appendix == appendix) & 
                     (results_df.motors == motors), :].iloc[0]

plot_name = f"plots/experiments/{exp_name}{appendix}_motors{motors}"

In [ ]:
def plot_positions(ax, positions_cm):
    from matplotlib import cm
    
    cmap = cm.get_cmap('inferno') 
    n_labels = 3
    label = None
    step = len(positions_cm) // n_labels
    for i, p in enumerate(positions_cm):
        if i % step == 0 or (i == len(positions_cm) - 1):
            label = f'position {i}'
        ax.scatter(*p[:2], color=cmap(i / len(positions_cm)), label=label)
        label=None
    ax.legend()
    ax.plot(positions_cm[:, 0], positions_cm[:, 1], color='k', ls=':')
    ax.axis('equal')
    ax.set_xlabel('x [cm]')
    ax.set_ylabel('y [cm]')

In [ ]:
from plotting_tools import pcolorfast_custom, FIGSIZE, save_fig
from simulation import get_df_theory

min_distance = 11
chosen_sweeps = np.where(row.positions[:, 2] > 0.3)[0]

positions_cm = get_positions(row, chosen_sweeps, min_distance, end_distance=end_distance)

fig, ax = plt.subplots()
fig.set_size_inches(FIGSIZE, FIGSIZE)
plot_positions(ax, positions_cm)
save_fig(fig, plot_name + "_positions.pdf")

freqs = row.frequencies_matrix[0, :]

distances = positions_cm[:, 1]

xvalues = distances; xlabel="distance [cm]"
#xvalues = row.seconds[chosen_sweeps]; xlabel="seconds [s]"

yvalues = freqs[freqs>0]
values = np.abs(row.stft[chosen_sweeps])[..., freqs>0]

sort_idx = np.argsort(xvalues)
xvalues = xvalues[sort_idx]
values = values[sort_idx, ...]

df = get_df_theory(yvalues, np.sort(distances))

calib_function = get_calib_function(calib_method)
calib_values = calib_function(yvalues)

values /= calib_values

mic_idx = 0

step = 500
yticks = np.arange(min(yvalues)//step*step, max(yvalues)//step*step+2*step, step=step)

fig, ax = plt.subplots()
fig.set_size_inches(2*FIGSIZE, FIGSIZE)
pcolorfast_custom(ax, np.arange(len(xvalues)), yvalues, values[:, mic_idx, :].T)
ax.set_xticklabels(np.round(xvalues, 1), rotation=90)
ax.set_yticks(yticks)
ax.set_yticklabels(yticks)
ax.set_ylabel("frequency [Hz]")
ax.set_xlabel(xlabel)

fig, ax = plt.subplots()
fig.set_size_inches(2*FIGSIZE, FIGSIZE)
pcolorfast_custom(ax, np.arange(len(xvalues)), yvalues, df[mic_idx, :, :])
ax.set_xticklabels(np.round(xvalues, 1), rotation=90)
ax.set_yticks(yticks)
ax.set_yticklabels(yticks)
ax.set_ylabel("frequency [Hz]")
ax.set_xlabel(xlabel)

In [ ]:
from plotting_tools import add_colorbar
from inference import Inference, eps_normalize
from estimators import DistanceEstimator

mics = range(2) #[0, 1, 2, 3]
inf_machine = Inference()

distance_estimator = DistanceEstimator()
dists = distance_estimator.DISTANCES_M * 1e2

results_matrix = np.empty((values.shape[0], len(dists)))
for i in range(values.shape[0]):
    inf_machine.add_data(values[i], yvalues)
    
    inf_machine.add_geometry([0, 80], WALL_ANGLE_DEG)
    for mic_i in mics: 
        dist, prob, diff = inf_machine.do_inference("bayes", mic_i)
        distance_estimator.add_distribution(diff*1e-2, prob, mic_i)
        
    d, prob = distance_estimator.get_distance_distribution()
    results_matrix[i, :] = prob #eps_normalize(prob, eps=1e-50)
    
fig, ax = plt.subplots()
fig.set_size_inches(2*FIGSIZE, FIGSIZE)
im = pcolorfast_custom(ax, np.arange(values.shape[0]), dists, np.log10(results_matrix.T), vmin=-50)
#im = pcolorfast_custom(ax, np.arange(values.shape[0]), dists, results_matrix.T)
ax.set_xticklabels(np.round(xvalues, 1), rotation=90)
ax.set_ylabel("estimated distance [cm]")
ax.set_xlabel("real distance [cm]")
ax.set_yticks(np.arange(min(dists), max(dists), step=10))
ax.set_yticklabels(np.arange(min(dists), max(dists), step=10).astype(int))
ax.plot(np.arange(values.shape[0])+0.5, xvalues, color="C1", label="ground truth")
ax.set_ylim(min(dists), max(dists))
ax.legend()
add_colorbar(fig, ax, im, title="log-probability")

argmaxs = np.argmax(results_matrix, axis=1)
ax.scatter(np.arange(values.shape[0])+0.5, dists[argmaxs], color="C1")
save_fig(fig, plot_name + "_probabilities.pdf")

maxs = np.max(results_matrix, axis=1)
fig, ax = plt.subplots()
fig.set_size_inches(2*FIGSIZE, FIGSIZE)
ax.plot(np.arange(values.shape[0]), maxs)
ax.set_xticks(np.arange(values.shape[0]))
ax.set_xticklabels(np.round(xvalues, 1), rotation=90)